<a href="https://colab.research.google.com/github/eliasakalu/ICogLabs_AI_Projects/blob/main/Classify_Number_1_to_9_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

In [3]:
#Data for training and testing
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

# Normalize
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

#Reshape the matrix to Vector
x_train,x_test=x_train.reshape(-1,28*28),x_test.reshape(-1,28*28)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
#Model Definingg
model=tf.keras.Sequential([
    tf.keras.layers.Dense(units=30,activation="relu",input_shape=(28*28,)),
    tf.keras.layers.Dense(units=15,activation="relu"),
    tf.keras.layers.Dense(units=10,activation="softmax")

])

model.compile(optimizer="adam",loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=["accuracy"])
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10)

Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7827 - loss: 0.7068 - val_accuracy: 0.9374 - val_loss: 0.2094
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9432 - loss: 0.1987 - val_accuracy: 0.9501 - val_loss: 0.1684
Epoch 3/10
1262/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9569 - loss: 0.1474

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
#Test the image uploaded
files=list(uploaded.keys())[0]
img=cv2.imread(files,cv2.IMREAD_GRAYSCALE)
img=img/255.0
img=1-img
img=cv2.resize(img,(28,28))
img=img.reshape(-1,28*28)
predicted_class=model.predict(img)
print(predicted_class)
index=np.argmax(predicted_class)
print(f"The model recognizes the number {index}")

In [ ]:
#Live video will not work in google colab but uploaded video can be
# Upload video
vid = files.upload()
vid = list(vid.keys())[0]

In [ ]:
cap = cv2.VideoCapture(vid)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Frame dimensions
    h, w = gray.shape

    # Define ROI (safe center square)
    size = min(200, h, w)
    x1 = max(0, w//2 - size//2)
    y1 = max(0, h//2 - size//2)
    roi = gray[y1:y1+size, x1:x1+size]

    # Preprocess for MNIST
    roi = cv2.resize(roi, (28, 28))
    roi = roi.astype("float32") / 255.0
    roi = 1.0 - roi
    roi = roi.reshape(1, 28*28)

    # Predict
    pred = model.predict(roi, verbose=0)
    digit = np.argmax(pred)

    # Draw ROI + text
    cv2.rectangle(frame, (x1, y1), (x1+size, y1+size), (0, 255, 0), 2)
    cv2.putText(
        frame,
        f"Prediction: {digit}",
        (10, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2
    )

    # Show frame in Colab
    cv2_imshow(frame)

cap.release()